# Sequence to Sequence model (Encoder and Decoder Technique)
Translate one language into another language (English to French)

In [1]:
import numpy as np 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input , LSTM , Dense

batch_size = 64 # batch size for training
epochs = 100 
latent_dims = 256 # latent dimensionality of the encoding space means - Feature dimension
num_samples = 10000 # number of samples to train on

data_path = "/content/drive/MyDrive/My Data/fra.txt"

Reading our input and target data and also read input character (@,$,) and target character

In [2]:
# Vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text , target_text, _ = line.split("\t")
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [3]:
input_characters

{' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é'}

In [4]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [5]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [6]:
# Assign tokens ( 0 ,1 , 2 so on) to each and every input and target characters

input_token_index = dict([(char , i) for i , char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [7]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 'é': 70}

In [8]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '$': 4,
 '%': 5,
 '&': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 ',': 10,
 '-': 11,
 '.': 12,
 '0': 13,
 '1': 14,
 '2': 15,
 '3': 16,
 '5': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 '?': 21,
 'A': 22,
 'B': 23,
 'C': 24,
 'D': 25,
 'E': 26,
 'F': 27,
 'G': 28,
 'H': 29,
 'I': 30,
 'J': 31,
 'K': 32,
 'L': 33,
 'M': 34,
 'N': 35,
 'O': 36,
 'P': 37,
 'Q': 38,
 'R': 39,
 'S': 40,
 'T': 41,
 'U': 42,
 'V': 43,
 'Y': 44,
 'a': 45,
 'b': 46,
 'c': 47,
 'd': 48,
 'e': 49,
 'f': 50,
 'g': 51,
 'h': 52,
 'i': 53,
 'j': 54,
 'k': 55,
 'l': 56,
 'm': 57,
 'n': 58,
 'o': 59,
 'p': 60,
 'q': 61,
 'r': 62,
 's': 63,
 't': 64,
 'u': 65,
 'v': 66,
 'w': 67,
 'x': 68,
 'y': 69,
 'z': 70,
 '\xa0': 71,
 '«': 72,
 '»': 73,
 'À': 74,
 'Ç': 75,
 'É': 76,
 'Ê': 77,
 'à': 78,
 'â': 79,
 'ç': 80,
 'è': 81,
 'é': 82,
 'ê': 83,
 'î': 84,
 'ï': 85,
 'ô': 86,
 'ù': 87,
 'û': 88,
 'œ': 89,
 '\u2009': 90,
 '’': 91,
 '\u202f': 92}

## One Hot Representation

In [9]:
encoder_input_data = np.zeros(
    (len(input_texts) , max_encoder_seq_length , num_encoder_tokens) , dtype="float32"
)

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [11]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0

    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0

        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
            
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [12]:
encoder_input_data.shape

(10000, 15, 71)

## Build a model

In [13]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None , num_encoder_tokens))
encoder = LSTM(latent_dims , return_state = True)
encoder_outputs , state_h , state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h , state_c]

In [14]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None , num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dims , return_sequences = True , return_state = True)
decoder_outputs , _, _ = decoder_lstm(decoder_inputs , initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens , activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [15]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`

model = Model([encoder_inputs , decoder_inputs] , decoder_outputs)

## Train a Model

In [16]:
model.compile(optimizer="rmsprop" , loss="categorical_crossentropy" , metrics=["accuracy"])

In [17]:
model.fit(
    [encoder_input_data , decoder_input_data],
    decoder_target_data,
    batch_size = batch_size,
    epochs=epochs,
    validation_split=0.2
)

Epoch 1/100
125/125 [==============================] - 36s 20ms/step - loss: 1.5234 - accuracy: 0.7028 - val_loss: 1.0430 - val_accuracy: 0.7097
Epoch 2/100
125/125 [==============================] - 1s 11ms/step - loss: 0.8865 - accuracy: 0.7602 - val_loss: 0.8396 - val_accuracy: 0.7716
Epoch 3/100
125/125 [==============================] - 1s 12ms/step - loss: 0.7141 - accuracy: 0.8063 - val_loss: 0.7011 - val_accuracy: 0.7994
Epoch 4/100
125/125 [==============================] - 1s 12ms/step - loss: 0.5935 - accuracy: 0.8283 - val_loss: 0.6426 - val_accuracy: 0.8127
Epoch 5/100
125/125 [==============================] - 2s 12ms/step - loss: 0.5382 - accuracy: 0.8431 - val_loss: 0.5938 - val_accuracy: 0.8281
Epoch 6/100
125/125 [==============================] - 2s 12ms/step - loss: 0.5033 - accuracy: 0.8528 - val_loss: 0.5614 - val_accuracy: 0.8362
Epoch 7/100
125/125 [==============================] - 1s 12ms/step - loss: 0.4672 - accuracy: 0.8626 - val_loss: 0.5423 - val_accuracy

In [18]:
# Save a model for Sampling and Prediction
model.save("s2s")

INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


## Run inference (sampling)
 - encode input and retrieve initial decoder state
 - run one step of decoder with this initial state and a "start of sequence" token as target. Output will be the next target token.
 - Repeat with the current target token and current states

In [19]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
from tensorflow.keras.models import load_model

model = load_model("s2s")

In [20]:
encoder_inputs = model.input[0] # input_1
encoder_inputs

<KerasTensor: shape=(None, None, 71) dtype=float32 (created by layer 'input_1')>

In [24]:
encoder_outputs , state_h_enc , state_c_snc = model.layers[2].output # lstm_1
encoder_states = [state_h_enc , state_c_snc]
encoder_model = Model(encoder_inputs , encoder_states)

In [25]:
decoder_inputs = model.inputs[1] # input_2
decoder_inputs

<KerasTensor: shape=(None, None, 93) dtype=float32 (created by layer 'input_2')>

In [26]:
decoder_state_input_h = Input(shape=(latent_dims) , name="input_3")
decoder_state_input_c = Input(shape=(latent_dims) , name="input_4")
decoder_states_inputs = [decoder_state_input_h , decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs , state_h_dec , state_c_dec = decoder_lstm(
    decoder_inputs , initial_state = decoder_states_inputs
)

In [27]:
decoder_states = [state_h_dec , state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)

In [28]:
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs , [decoder_outputs] + decoder_states
)

In [29]:
# Reverse-lookup token index to decode sequences back to
# something readable.

reverse_input_char_index = dict((i , char) for char , i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

reverse_input_char_index

{0: ' ',
 1: '!',
 2: '"',
 3: '$',
 4: '%',
 5: '&',
 6: "'",
 7: ',',
 8: '-',
 9: '.',
 10: '0',
 11: '1',
 12: '2',
 13: '3',
 14: '5',
 15: '7',
 16: '8',
 17: '9',
 18: ':',
 19: '?',
 20: 'A',
 21: 'B',
 22: 'C',
 23: 'D',
 24: 'E',
 25: 'F',
 26: 'G',
 27: 'H',
 28: 'I',
 29: 'J',
 30: 'K',
 31: 'L',
 32: 'M',
 33: 'N',
 34: 'O',
 35: 'P',
 36: 'Q',
 37: 'R',
 38: 'S',
 39: 'T',
 40: 'U',
 41: 'V',
 42: 'W',
 43: 'Y',
 44: 'a',
 45: 'b',
 46: 'c',
 47: 'd',
 48: 'e',
 49: 'f',
 50: 'g',
 51: 'h',
 52: 'i',
 53: 'j',
 54: 'k',
 55: 'l',
 56: 'm',
 57: 'n',
 58: 'o',
 59: 'p',
 60: 'q',
 61: 'r',
 62: 's',
 63: 't',
 64: 'u',
 65: 'v',
 66: 'w',
 67: 'x',
 68: 'y',
 69: 'z',
 70: 'é'}

In [30]:
def decode_sequence(input_seq):
  # Encode the input as state vectors.
  states_value = encoder_model.predict(input_seq)

  # Generate empty target sequence of length 1.
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  # Populate the first character of target sequence with the start character.
  target_seq[0, 0, target_token_index["\t"]] = 1.0

  # Sampling loop for a batch of sequences
  # (to simplify, here we assume a batch of size 1).
  stop_condition = False
  decoded_sentence = ""
  while not stop_condition:
      output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

      # Sample a token
      sampled_token_index = np.argmax(output_tokens[0, -1, :])
      sampled_char = reverse_target_char_index[sampled_token_index]
      decoded_sentence += sampled_char

      # Exit condition: either hit max length
      # or find stop character.
      if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
          stop_condition = True

      # Update the target sequence (of length 1).
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      target_seq[0, 0, sampled_token_index] = 1.0

      # Update states
      states_value = [h, c]
  return decoded_sentence

You can now generate decoded sentences as such:

In [31]:
for seq_index in range(20):
  # Take one sequence (part of the training set)
  # for trying out decoding.
  input_seq = encoder_input_data[seq_index : seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print("-")
  print("Input Sentence:-", input_texts[seq_index])
  print("Decoded Sentence:-", decoded_sentence)

-
Input Sentence:- Go.
Decoded Sentence:- Marche.

-
Input Sentence:- Go.
Decoded Sentence:- Marche.

-
Input Sentence:- Go.
Decoded Sentence:- Marche.

-
Input Sentence:- Hi.
Decoded Sentence:- Salut.

-
Input Sentence:- Hi.
Decoded Sentence:- Salut.

-
Input Sentence:- Run!
Decoded Sentence:- Cours !

-
Input Sentence:- Run!
Decoded Sentence:- Cours !

-
Input Sentence:- Run!
Decoded Sentence:- Cours !

-
Input Sentence:- Run!
Decoded Sentence:- Cours !

-
Input Sentence:- Run!
Decoded Sentence:- Cours !

-
Input Sentence:- Run!
Decoded Sentence:- Cours !

-
Input Sentence:- Run!
Decoded Sentence:- Cours !

-
Input Sentence:- Run!
Decoded Sentence:- Cours !

-
Input Sentence:- Run.
Decoded Sentence:- Cours !

-
Input Sentence:- Run.
Decoded Sentence:- Cours !

-
Input Sentence:- Run.
Decoded Sentence:- Cours !

-
Input Sentence:- Run.
Decoded Sentence:- Cours !

-
Input Sentence:- Run.
Decoded Sentence:- Cours !

-
Input Sentence:- Run.
Decoded Sentence:- Cours !

-
Input Sentence:- 